In [8]:
import pandas as pd
import urllib.parse
import re

In [9]:
df = pd.read_csv('/kaggle/input/network-ptmxh/edges_raw.csv')

print(f"Số dòng ban đầu: {len(df)}")

Số dòng ban đầu: 19935


In [10]:
# 1. Giải mã URL trước (bắt buộc)
df['src'] = df['src'].apply(lambda x: urllib.parse.unquote(str(x)))
df['des'] = df['des'].apply(lambda x: urllib.parse.unquote(str(x)))

In [11]:
def clean_wiki_entity(text):
    s = str(text).strip()
    
    # Bỏ hết tiền tố Wikipedia thừa
    s = re.sub(r'^/wiki/', '', s)                    # bỏ /wiki/
    s = re.sub(r'^https?://\w+\.wikipedia\.org/wiki/', '', s)
    
    # Xử lý redlink (trang không tồn tại)
    s = re.sub(r'w/index\.php\?title=', '', s)
    s = re.sub(r'&action=edit&redlink=1', '', s)
    s = re.sub(r'\(không có trang\)', '', s)         # nếu có rồi thì bỏ
    
    # Xử lý các trường hợp "không rõ"
    if s in ['Doctoral_advisor', 'https://en.wikipedia.org/wiki/Doctoral_advisor']:
        return "Không có người hướng dẫn tiến sĩ"
    if 'thesis.library.caltech.edu' in s or 'archive.org' in s:
        return "Luận án tiến sĩ"
    
    # Chuẩn hóa dấu cách và dấu gạch dưới
    s = s.replace('_', ' ')
    
    # Viết hoa chữ cái đầu mỗi từ (title case đẹp hơn)
    s = re.sub(r'\b(\w)', lambda m: m.group(1).upper(), s)
    s = re.sub(r'\s+', ' ', s)  # bỏ khoảng trắng thừa
    
    # Loại bỏ dấu câu thừa ở cuối
    s = s.strip(' ,.()[]')
    
    return s.strip()

In [12]:
print("Đang làm sạch tên thực thể...")
df['src'] = df['src'].apply(clean_wiki_entity)
df['des'] = df['des'].apply(clean_wiki_entity)

# Làm sạch cột type (chỉ trim + chuẩn hóa)
df['type'] = df['type'].str.strip()

# Loại bỏ dòng trùng + sắp xếp
df.drop_duplicates(inplace=True)
df.sort_values(['src', 'des', 'type'], inplace=True)
df.reset_index(drop=True, inplace=True)

Đang làm sạch tên thực thể...


In [13]:
# Làm sạch cột type (chỉ trim + chuẩn hóa)
df['type'] = df['type'].str.strip()


In [14]:
# Loại bỏ dòng trùng + sắp xếp
df.drop_duplicates(inplace=True)
df.sort_values(['src', 'des', 'type'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [18]:
# Lưu
df.to_csv('edges_cleaned.csv', index=False, encoding='utf-8')

print(f"HOÀN TẤT! Còn lại {len(df):,} dòng sạch đẹp")

HOÀN TẤT! Còn lại 10,257 dòng sạch đẹp
